In [6]:
from textblob import TextBlob
import tweepy
import twitter_credentials as tc
import sys
import pandas as pd

#auth
auth = tweepy.OAuthHandler(tc.API_KEY, tc.API_KEY_SECRET)
auth.set_access_token(tc.ACCESS_TOKEN, tc.ACCESS_TOKEN_SECRET)

api = tweepy.API(auth)

In [7]:
search_term = 'world cup 2022'
tweet_amount = 20

tweets = tweepy.Cursor(method=api.search_tweets, q=search_term, lang='en').items(tweet_amount)

In [9]:
polarity = 0

positive = 0
negative = 0
neutral = 0

tweets_clean = []
for tweet in tweets:
    tweet_text = tweet.text.replace('RT', '')       #Remove the RT Tags
    if tweet_text.startswith(' @'):                  #Remove usernames
        position = tweet_text.index(':')
        tweet_text = tweet_text[position+2:]
        tweets_clean.append(tweet_text)
    if tweet_text.startswith('@'):                  #Remove usernames
        position = tweet_text.index(' ')
        tweet_text = tweet_text[position+2:]
        tweets_clean.append(tweet_text)
    #print(tweet_text)
    #Analysis
    analysis = TextBlob(tweet_text)
    tweet_polarity = analysis.polarity
    polarity += tweet_polarity
    if tweet_polarity>0:
        positive +=1
    elif tweet_polarity<0:
        negative+=1
    else:
        neutral+=1

print(polarity)                            # We obtained A polarity of +54 which means the overal sentiment is very positive over the 200 tweets
print("positive tweets : ",positive)
print("negative tweets : ",negative)
print("neutral tweets : ",neutral)


3.5763636363636366
positive tweets :  11
negative tweets :  2
neutral tweets :  7


In [10]:
print(tweets_clean)

['Qatar hosted FIFA World Cup 2022 and gifted #KashmirPashmina shawls to VIPs.\nBrand Kashmir is soaring high as these shawls…', 'Emmy_Wise10 @_AsiwajuLerry Bro just delete this, Which Most world cup assist? Most complete player… https://t.co/AP1DMmpx5e', 'jungkook graduated from uni with high honors, released 4 solo songs and an anthology album with bts, got invited to the whi…', 'Katie McCabe leading Ireland to the 2023 World Cup was truly one of my highlights of 2022 https://t.co/kWWSTWSWWe', 'Soccer legend Pelé died on Thursday at the age of 82.\n\nFor over 60 years Pelé stood alone as Brazil’s all-time leading s…', 'Only 3 players made 20+ tackles at the 2022 World Cup, 2 of those were Enzo Fernandez and Mateo Kovacic (@Squawka) https://t.co…', 'Lionel Messi and his 2022 World Cup. Incredible. https://t.co/YQC9lJ3y0J', 'Two disasters, one near disaster and a World Cup with a body count of thousands. Meanwhile, another faultless year for…', '🕯️ Remembering Pelé 🕯️\n\nTributes flood

In [12]:
all_sentences = []
for word in tweets_clean:
    all_sentences.append(word)
    
all_sentences

lines = list()
for line in all_sentences:    
    words = line.split()
    for w in words: 
       lines.append(w)


print(lines)


['Qatar', 'hosted', 'FIFA', 'World', 'Cup', '2022', 'and', 'gifted', '#KashmirPashmina', 'shawls', 'to', 'VIPs.', 'Brand', 'Kashmir', 'is', 'soaring', 'high', 'as', 'these', 'shawls…', 'Emmy_Wise10', '@_AsiwajuLerry', 'Bro', 'just', 'delete', 'this,', 'Which', 'Most', 'world', 'cup', 'assist?', 'Most', 'complete', 'player…', 'https://t.co/AP1DMmpx5e', 'jungkook', 'graduated', 'from', 'uni', 'with', 'high', 'honors,', 'released', '4', 'solo', 'songs', 'and', 'an', 'anthology', 'album', 'with', 'bts,', 'got', 'invited', 'to', 'the', 'whi…', 'Katie', 'McCabe', 'leading', 'Ireland', 'to', 'the', '2023', 'World', 'Cup', 'was', 'truly', 'one', 'of', 'my', 'highlights', 'of', '2022', 'https://t.co/kWWSTWSWWe', 'Soccer', 'legend', 'Pelé', 'died', 'on', 'Thursday', 'at', 'the', 'age', 'of', '82.', 'For', 'over', '60', 'years', 'Pelé', 'stood', 'alone', 'as', 'Brazil’s', 'all-time', 'leading', 's…', 'Only', '3', 'players', 'made', '20+', 'tackles', 'at', 'the', '2022', 'World', 'Cup,', '2', 'of'